In [1]:
# Import necessary libraries
import os
from vllm import LLM, SamplingParams
from vllm.steer_vectors.request import SteerVectorRequest, VectorConfig
from transformers import AutoTokenizer

# Set environment variables
os.environ["VLLM_USE_V1"] = "0"
os.environ["CUDA_VISIBLE_DEVICES"] = "5"

# Initialize LLM with steering vector capability
llm = LLM(
    model="/data/zju-46/shenyl/hf/model/Qwen/Qwen2.5-1.5B-Instruct/",
    enable_steer_vector=True,
    enforce_eager=True,
    tensor_parallel_size=1
)

INFO 07-16 17:29:30 [__init__.py:244] Automatically detected platform cuda.
INFO 07-16 17:29:44 [config.py:841] This model supports multiple tasks: {'classify', 'embed', 'generate', 'reward'}. Defaulting to 'generate'.
INFO 07-16 17:29:44 [config.py:1472] Using max model len 32768
WARNING 07-16 17:29:45 [cuda.py:102] To see benefits of async output processing, enable CUDA graph. Since, enforce-eager is enabled, async output processor cannot be used
INFO 07-16 17:29:45 [llm_engine.py:232] Initializing a V0 LLM engine (v0.1.dev7499+g2a4b294) with config: model='/data/zju-46/shenyl/hf/model/Qwen/Qwen2.5-1.5B-Instruct/', speculative_config=None, tokenizer='/data/zju-46/shenyl/hf/model/Qwen/Qwen2.5-1.5B-Instruct/', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config={}, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=32768, download_dir=None, load_format=auto, tensor_parallel_size=1, pipeline_parallel_size=1, disable_cust

Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


INFO 07-16 17:29:47 [default_loader.py:272] Loading weights took 1.02 seconds
INFO 07-16 17:29:48 [model_runner.py:1255] Model loading took 2.8876 GiB and 1.183316 seconds
INFO 07-16 17:29:50 [worker.py:295] Memory profiling takes 1.46 seconds
INFO 07-16 17:29:50 [worker.py:295] the current vLLM instance can use total_gpu_memory (47.44GiB) x gpu_memory_utilization (0.90) = 42.69GiB
INFO 07-16 17:29:50 [worker.py:295] model weights take 2.89GiB; non_torch_memory takes 0.06GiB; PyTorch activation peak memory takes 2.02GiB; the rest of the memory reserved for KV Cache is 37.72GiB.
INFO 07-16 17:29:50 [executor_base.py:115] # cuda blocks: 88295, # CPU blocks: 9362
INFO 07-16 17:29:50 [executor_base.py:120] Maximum concurrency for 32768 tokens per request: 43.11x
INFO 07-16 17:29:54 [llm_engine.py:430] init engine (profile, create kv cache, warmup model) took 5.75 seconds


In [2]:
prompt_no_input_template = "<|im_start|>user\n%s<|im_end|>\n<|im_start|>assistant\n"
instruction = "Who are you?"
instruction2 = "Who am i?"
prompt = [prompt_no_input_template % instruction, prompt_no_input_template % instruction2]
# Generate baseline response without steering
example_answers = llm.generate(
    prompt,
    SamplingParams(
        temperature=0,
        max_tokens=16,
        skip_special_tokens=False,
    ),
)

# Display baseline response
print("=====Baseline=====")
print(example_answers[0].outputs[0].text)
print(example_answers[1].outputs[0].text)
# Configure steering vector request for emoji_loreft

sv_request = SteerVectorRequest(
    "emoji_loreft",
    1,
    steer_vector_local_path="./weight/",
    scale=1.0,
    target_layers=[8],
    algorithm="loreft",
    prefill_trigger_positions=[-1],
)
# Generate response with emoji_loreft
output = llm.generate(
    prompt, 
    SamplingParams(
        temperature=0,
        max_tokens=16,
        skip_special_tokens=False,
    ), 
    steer_vector_request=sv_request
)

# Display emoji_loreft response
print("=====Emoji_loreft Steered=====")
print(output[0].outputs[0].text)
print(output[1].outputs[0].text)

Adding requests:   0%|          | 0/2 [00:00<?, ?it/s]

Processed prompts:   0%| | 0/2 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, 

=====Baseline=====
I am an artificial intelligence designed to assist with various tasks and answer questions to the
You are a large language model.


Adding requests:   0%|          | 0/2 [00:00<?, ?it/s]

Processed prompts:   0%| | 0/2 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, 

=====Emoji_loreft Steered=====
🤖💬🌐🧠
👤❓🔍🌟
